In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from datasets import load_dataset
dataset= load_dataset('mteb/tweet_sentiment_extraction')
df = pd.DataFrame(dataset['train'])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [3]:
df.head()

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 26732
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3432
    })
})

In [5]:
dataset['train'][:8]

{'id': ['cb774db0d1',
  '549e992a42',
  '088c60f138',
  '9642c003ef',
  '358bd9e861',
  '28b57f3990',
  '6e0c6d75b1',
  'e050245fbd'],
 'text': [' I`d have responded, if I were going',
  ' Sooo SAD I will miss you here in San Diego!!!',
  'my boss is bullying me...',
  ' what interview! leave me alone',
  ' Sons of ****, why couldn`t they put them on the releases we already bought',
  'http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth',
  '2am feedings for the baby are fun when he is all smiles and coos',
  ' Both of you'],
 'label': [1, 0, 0, 0, 0, 1, 2, 1],
 'label_text': ['neutral',
  'negative',
  'negative',
  'negative',
  'negative',
  'neutral',
  'positive',
  'neutral']}

In [15]:
dataset['train'][:8]['text'], dataset['train'][:8]['label'], dataset['train'][:8]['label_text']

([' I`d have responded, if I were going',
  ' Sooo SAD I will miss you here in San Diego!!!',
  'my boss is bullying me...',
  ' what interview! leave me alone',
  ' Sons of ****, why couldn`t they put them on the releases we already bought',
  'http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth',
  '2am feedings for the baby are fun when he is all smiles and coos',
  ' Both of you'],
 [1, 0, 0, 0, 0, 1, 2, 1],
 ['neutral',
  'negative',
  'negative',
  'negative',
  'negative',
  'neutral',
  'positive',
  'neutral'])

In [9]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [19]:
sample_set = dataset['train'][10:50]
for txt,lab,lab_txt in zip(sample_set['text'],sample_set['label'],sample_set['label_text']):
    print(f'text:{txt} ------| label:{lab}---------| label_texts:{lab_txt}' )

text:I really really like the song Love Story by Taylor Swift ------| label:2---------| label_texts:positive
text:My Sharpie is running DANGERously low on ink ------| label:0---------| label_texts:negative
text:i want to go to music tonight but i lost my voice. ------| label:0---------| label_texts:negative
text:test test from the LG enV2 ------| label:1---------| label_texts:neutral
text:Uh oh, I am sunburned ------| label:0---------| label_texts:negative
text: S`ok, trying to plot alternatives as we speak *sigh* ------| label:0---------| label_texts:negative
text:i`ve been sick for the past few days  and thus, my hair looks wierd.  if i didnt have a hat on it would look... http://tinyurl.com/mnf4kw ------| label:0---------| label_texts:negative
text:is back home now      gonna miss every one ------| label:0---------| label_texts:negative
text:Hes just not that into you ------| label:1---------| label_texts:neutral
text: oh Marly, I`m so sorry!!  I hope you find her soon!! <3 <3 -----

In [20]:
# choosing gpt2 model 
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [21]:
def tokenize_function(examples):
    return tokenizer(examples['text'],padding='max_length',truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched = True)


Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [22]:
print(tokenized_dataset['train'])

Dataset({
    features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 26732
})


In [23]:
print(tokenized_dataset['train'][0])

{'id': 'cb774db0d1', 'text': ' I`d have responded, if I were going', 'label': 1, 'label_text': 'neutral', 'input_ids': [314, 63, 67, 423, 7082, 11, 611, 314, 547, 1016, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [24]:
print(dataset['train'][0].keys())

dict_keys(['id', 'text', 'label', 'label_text'])


In [26]:
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

In [30]:
small_train_dataset['text']

Column(['check on ya`ll in a bit. im in for a Mother`s day haircut. ciao!...', 'i need to write really bad. can someone go get my diary from my flat for me', 'peeling so much!!   ..I feel so bad', '_sm I`ll have a look. Do you live in the centre of Glasgow? I was born on Bellshill & grew up in Bothwell. I miss Tunnocks pies', ' hey you ****!! that`s kinda mean dont yu think?!'])

In [31]:
small_eval_dataset['text']

Column(['Lots of tasks to complete today. First, wecloming new members to Canadamigos Social Networking site, www.canadamigos.com, and coffee', 'Guess what? mom adopted a kitty today (11 months) His name is Corky', ' **** i shouldnt have left!', 'looking at the sunshine out of my office window', 'is 500 words into a 10 minute speech on 80s teen movies. due wednesday. blah blah. can`t wait for the glenn ball!'])

In [35]:
# intializing the base model
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# !pip install evaluate

In [42]:
#evaluating the model
import evaluate
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [43]:
# finetuning using trainer method
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    # evaluation_strategy='epoch',
    output_dir = 'test_trainer', per_device_train_batch_size=1,
    per_device_eval_batch_size=1,gradient_accumulation_steps=4 )

trainer= Trainer(model = model,args=training_args,
                train_dataset = small_train_dataset,
                eval_dataset=small_eval_dataset,compute_metrics=compute_metrics)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

In [ ]:
import evaluate 
trainer.evaluate()